In [3]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

In [4]:
# TODO: get_distance_matrix 
"""
params : 
 feature_vector: array of feature vectors for each shot 
return : 2D distance matrix (Matrix 2D with size N*N ,N is the numer of shots,cell (i,j) have the feature distance between i,j shots)
"""
features_matrix =  np.array([[2, 1, 0, 4, 1], [0, 3, 1, 1, 10],[1, 1, 0, 1, 0]])
def get_distance_matrix(features_matrix):
    D_Matrix = sparse.csr_matrix(features_matrix)
    D_Matrix = cosine_similarity(D_Matrix)
    D_Matrix = 1-D_Matrix
    # D_Matrix.tolist()
    return D_Matrix

#TODO: try eculedian  distance 

In [5]:
D_Matrix = get_distance_matrix(features_matrix)
print(D_Matrix)
print(1-D_Matrix)

[[ 2.22044605e-16  6.55986020e-01  1.38359563e-01]
 [ 6.55986020e-01 -2.22044605e-16  7.80801350e-01]
 [ 1.38359563e-01  7.80801350e-01 -2.22044605e-16]]
[[1.         0.34401398 0.86164044]
 [0.34401398 1.         0.21919865]
 [0.86164044 0.21919865 1.        ]]


In [6]:
#TODO: get_internal_sums
"""get the total sum of distance of different segmentation 
params :
 Distance_Matrix: 2D distance matrix 
 N: number of shots 
return : table with all different segments in D_Matrix
"""
def get_internal_sums(D_Matrix,N):
    # D_sum = torch.zeros(N,N, device=self.device)
    D_sum = np.zeros(shape=(N,N))
    # initialize diagonal 
    for shot_index in range(N):
        D_sum[shot_index,shot_index] = D_Matrix[shot_index,shot_index]
    # initialize second diagonal
    for shot_index in range(0, N-1):
        D_sum[shot_index, shot_index+1] = D_Matrix[shot_index:shot_index+1+1, shot_index:shot_index+1+1].sum()
        print(D_sum[shot_index, shot_index+1])
        # TODO: recheck this condition 
        D_sum[shot_index+1, shot_index] = D_sum[shot_index, shot_index+1]
        print(D_sum[shot_index+1, shot_index])
    for scene_size in range(2, N):
        for start_shot in range(0, N - scene_size):
            '''
            D_sum[i,j] =+D_sum[i-1,j]                  --> missing last row
                        +D_sum[i,j-1]                  --> missing last column 
                        -D_sum[i-1,j-1]                --> as we sum it twice before
                        +D_Matrix[i,j] + D_Matrix[j,i] -->missing cells in brevious calculations  
            '''
            D_sum[start_shot, start_shot + scene_size] = D_sum[start_shot, start_shot + scene_size - 1] + D_sum[start_shot - 1, start_shot + scene_size] - D_sum[start_shot - 1, start_shot + scene_size - 1] +  D_Matrix[start_shot, start_shot + scene_size] + D_Matrix[start_shot + scene_size, start_shot] 
            # as the matrix is symetric 
            D_sum[start_shot + scene_size, start_shot] = D_sum[start_shot, start_shot + scene_size]
    return D_sum

In [7]:
D_sum = get_internal_sums(D_Matrix,3)
print(D_sum)

1.3119720402611001
1.3119720402611001
1.5616027005190467
1.5616027005190467
[[ 2.22044605e-16  1.31197204e+00  2.70884660e-02]
 [ 1.31197204e+00 -2.22044605e-16  1.56160270e+00]
 [ 2.70884660e-02  1.56160270e+00 -2.22044605e-16]]


In [8]:
from operator import itemgetter
# TODO: hierarchical clustering_h_add
# TODO: NOW I AM WORKING WITHOUT USING INTERMEDIATE SUMS (I SHOULD USE IT LATER FOR EFFECIENCY)
"""params : 
    D_matrix : 2D distance matrix (with size N*N)
    scenes_num: Nuber of scenes needed to be generated 
return : T (T is the optimal shot indexes where thw shots are devided into scenes)
"""
def get_optimal_sequence_add(D_matrix, scenes_num):
    N = D_Matrix.shape[0]
    '''
    define cost matrix to define the optimal objective function value ,the optimal one is then cost_Matrix[1,scenes_num]
    cost_Matrix[n,scenes_num] means the cost of deviding  N-n+1 shots to scenes_num scenes
    '''
    cost_Matrix = np.zeros((N, scenes_num))
    boundry_index_Matrix = np.zeros((N, scenes_num), dtype=int)

    # initialization
    for n in range(0, N):
        cost_Matrix[n, 0] = np.sum(D_matrix[n:, n:])

    for n in range(0, N):
        boundry_index_Matrix[n, 0] = N - 1

    # build the rest of the table 
    for k in range(1,scenes_num):
        for n in range(0,N):
            diffrent_scene_costs = []
            # define i the index of the next boundry will be betwen (n,N)
            for i in range(n, N):
                if i < N - 1:
                    C_prev = cost_Matrix[i + 1, k - 1]
                else:
                    C_prev = 0
                scene_cost = np.sum(D_matrix[n:i + 1, n:i + 1])
                scene_cost = scene_cost + C_prev
                diffrent_scene_costs.append((scene_cost,i))
            # diffrent_scene_costs = np.array(diffrent_scene_costs)
            # choose the minimum value that we can put i and achieve the least cost for the scene 
            # cost_Matrix[n, k] = np.min(diffrent_scene_costs)
            min_cost = min(diffrent_scene_costs, key=itemgetter(0))
            cost_Matrix[n, k] = min_cost[0]
            # save the index
            # boundry_index_Matrix[n, k] = np.where(diffrent_scene_costs == cost_Matrix[n, k])[0][0] + n
            boundry_index_Matrix[n, k] = min_cost[0]+n
    
    scene_boundries = np.zeros((scenes_num,), dtype=int)
    scene_boundries_prev = 0
    for i in range(0, scenes_num):
        if i == 0:
            scene_boundries_prev = 0
        else:
            scene_boundries_prev = scene_boundries[i - 1]
        scene_boundries[i] = boundry_index_Matrix[scene_boundries_prev, scenes_num - i - 1]
    return scene_boundries

In [9]:
g = get_optimal_sequence_add(D_Matrix, 2)
print(g)

[1 2]


In [10]:
# TODO: hierarchical clustering_h_nrm
"""
params : 2D distance matrix (with size N*N)
return : T (T is the optimal shot indexes where thw shots are devided into scenes)
"""


'\nparams : 2D distance matrix (with size N*N)\nreturn : T (T is the optimal shot indexes where thw shots are devided into scenes)\n'

In [11]:
# TODO: get estimated number of scenes